# 半监督学习

之前的章节，我们介绍的都是有监督的学习，它的特点是每一个样本都带有对应的标签，标签值是我们关心的最终的答案，我们一般称它为Groud Truth。但当我们想去应用机器学习时，我们会发现，我们通过各类传感器每天会产生大量的数据，但是这些数据往往都是没有标签的。一般的做法是，我们采用人工标注的办法，但人工标注有两方面的问题：１）成本很好，而且越来越高；２）机器学习在很多方面的应用中是超过人的，所以人工标注的数据往往是不可靠的，需要进一步清洗。

所以如何利用这些没有标签的数据呢？半监督学习就是方法之一，后面我们还会介绍到其他无监督的学习方法。

>这里需要一张图，来说明label数据很少，而unlabel数据很多。

## 1. 什么是半监督学习

半监督学习中的unlabeled data与有标签的数据是同源图片，比如标签数据是猫和狗的图片，我们要做分类，那无标签的数据也是猫和狗。

![](https://note.youdao.com/yws/public/resource/c78ad83387bcb7adf414b412754f03ad/xmlnote/WEBRESOURCE9980aca4676ae4d703936ee6d2a04ded/19118)

## 2. 为什么半监督学习能够work

为什么unlabed的data能够对我们的学习模型有帮助？

我们利用unlabeled data往往是因为我们对数据的分布有一定的假设，然后通过这些Unlabeded data我们可以更加精确的把我们假想的分布参数求出来。

## 3. 应用在概率生成模型中

在之前介绍过概率生成模型，我们通过对每个类别建一个概率分布，来算样本属于某个类别的概率。

有了unlabeled的data后，我们可以得到这些unlabeled data属于每一类的概率值，然后我们将这些Unlabled data以概率值为权重对每个类别的分布的参数进行更新。

这样每个类别的分布参数就会根据unlabled data进行调整了。

具体到新$\mu$和$\Sigma$怎么算法的，还是利用最大似然，结果是很符合直觉的。

其实上面的步骤就是E-M算法，可以再结合GMM进行具体的公式推导。

## 4. Self Training

这个做法非常的简单，就是利用label data训练分类器，然后对unlabeled data进行分类，这些分类的结果可以直接作为unlabeled data的标签，然后把这些数据加入到训练集中，训练模型。

这些新加的unlabeled data在训练时对loss的贡献可以设置的小一些。或者根据unlabled data分类的confidence给一个权重。

为什么self training可以work呢，是因为我们的分类器，有时候虽然能把一些样本分类对，但confidence可能并不高，Loss并不为零。说明分类器还有进步的空间。

所以self Training对回归问题是没有作用的。

## 5. 基于Entropy正则化

我们的假设前提是：“非黑即白”，也就是我们的分类任务，各个类别之间是没有联系的，属于类别A，就不会属于类别B。

这样的思想指导我们，对于分类模型的输出的一个vector，它的value是属于各个类别的概率值。我们希望这个vector有一个值特别大，其他最好都是0，而不是有几个都很大，或者都差不多大。后者的情况，所明这个分类器没有Train好。

也就是说我们希望得到**清晰的分类**，那么如何定义清晰分类呢。

所以我们会计算每个样本预测出来结果vector的Entropy，我们希望它越大越好，所以我们取负号后，加到我们的损失函数里，充当正则项。

$$L = \sum_{x^{(r)}}C(y^{(r)},\hat{y}^{(r)})+\lambda\sum_{x^{(u)}}E(y^{(u)})$$

左边那项是针对labeled data的，后面那项是针对unlabeled data的。

## 6. Semi-supervised SVM

对unabled data可能属于的label进行穷举，然后每种情况，都训练一个SVM，求最后的margin和error，最后在这么多穷举里，选择一个margin最大，error最小的分类。

听起来这个算法实现上完全不可行，这计算量得有多大啊。但有一些方法，可以减少这些穷举的可能。

Thorsten Joachims, ”Transductive Inference for Text Classification using Support Vector Machines”, ICML, 1999

## 7. Smoothness Assumption

所有观测到的现象，都是连续平滑的，相近的输入，会产生相近的输出。
> "近朱者赤，近墨者黑"

上面的结论真的成立吗？实际需要加假设：

- x的分布是不平均的
- 如果样本$x^1$和$x^2$在一个高密度的空间区域上是靠近(**通过一个high densiy相连**)的，则它们的预测值$y^1$和$y^2$就会是相同的。

![](https://note.youdao.com/yws/public/resource/c78ad83387bcb7adf414b412754f03ad/xmlnote/WEBRESOURCEce7231a88e80776de11023c530092122/19103)

![](https://note.youdao.com/yws/public/resource/c78ad83387bcb7adf414b412754f03ad/xmlnote/WEBRESOURCEbefae5cdb87a04281c2b2b2ad8ec1c4f/19107)

对于上面这张图，我们初看，或者让我们的分类器看，我们会觉得中间那个2和后面的3近接近，而和最左边的2是不太像的，因为写法上就不同。但如果我们再看下面这张照片，我们就会觉得它到底还是2。

![](https://note.youdao.com/yws/public/resource/c78ad83387bcb7adf414b412754f03ad/xmlnote/WEBRESOURCEded50e4420895746bbd642be5786b046/19109)

这件事情可以类比到其他的识别任务里去，比如：我们的人脸识别系统往往对正脸和侧脸之间的相似度判断的并不好，但是如果我们有一系列的正脸到侧脸之间的角度的照片，那我们就可以用这种相似的传递关系，把正脸和侧脸关联起来。

![](https://note.youdao.com/yws/public/resource/c78ad83387bcb7adf414b412754f03ad/xmlnote/WEBRESOURCE06648edd4810c09ce5a92e3248686d67/19111)

## 8. 利用聚类

这种方法，也很常见了，在人脸识别中也经常用，比如：我们有某个地方10万个人的证件照，每个照片都有身份证号来作为他们的类别，而对于摄像头采集到的一系列抓拍图，我们是没有label的，我们就可以利用聚类算法，将这些unlabeled的人脸抓拍图与证件照在一起聚类，然后每个unlabeled data的label就用它属于的那个类的证件归照作为label就可以了。

![](https://note.youdao.com/yws/public/resource/c78ad83387bcb7adf414b412754f03ad/xmlnote/WEBRESOURCEb3d94db2b4508a7e71847aed23f68c17/19113)

## 9 Graph-Based Approach

怎么知道高维空间分布中的两个点$x^1$和$x^2$是通过一个high density path相连的呢。

我们可以构建一个图，根据图的连通来判断两个点是不是通过high density path相连。

每个样本都表示为图中的一个结点。然后通过定义一个样本与样本之间距离的度量函数$S(x^i,x^j)$，可以使用Gaussian Radial Basis Function。也可以使用余弦距离或欧式距离。

$$S(x^i, x^j) = \exp(-\gamma||x^i-x^j||^2)$$

然后每个结点通过求topK来看它的邻居。这样就形成了很多等价对，然后我们使用Find-Union算法，就可以形成很多连通图。这实际就是基于DBSCAN的聚类算法。

但这个算法work的前提是data要够多，不然连通图就连不起来了。

![](https://note.youdao.com/yws/public/resource/c78ad83387bcb7adf414b412754f03ad/xmlnote/WEBRESOURCEb19c1ec2c3c2f32a7b358588247f3651/19115)

## 10. 基于label相似的正则项

这部分没大看明白，大意是先对样本按第9小节描述的进行建图，那完后得到每两个样本之间的权重，也就是相似度，相似度越近，我们希望它们的输出越像。所以对最后的输入$y$，我们可以在Loss上针对这个点来加正则项。

$$S=\frac{1}{2}\sum_{i,j}(y^i-y^j)^2 = \mathbf{y}^TL\mathbf{y}$$
$$L=\sum_{x^r}C(y^r,\hat{y}^r)+\lambda S$$

> TODO　可以把这块的视频再听一遍！